### Doc 360 Call Activity

In [1]:
import polars as pl
import gc
import pandas as pd
import json

In [2]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)
roster_file = js['roster_file']
num_weeks_calls = js['num_weeks_calls']
bucket = js['bucket']

dflib = f's3://{bucket}/BIT/dataframes/'
rst = f's3://{bucket}/BIT/roster/'

In [3]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

In [4]:
# Importing Dependencies
load('temp_calls')
load('temp_abbv')
roster = (
    pl.read_parquet(f'{rst}MasterRoster_{roster_file}.parquet')
    .with_columns(
        pl.col('SalesRepIID').cast(pl.Int64),
        Rep = pl.col('EmpFName')+' '+pl.col('EmpLName')
    ).select(['SalesRepIID','Rep'])
)

#fixes for vortex import -> Probably caused by Polars Upgrades
temp_calls = temp_calls.with_columns(pl.col('SalesRepIID').cast(pl.Int64))
temp_abbv = temp_abbv.with_columns(pl.col('SalesRepIID').cast(pl.Int64))


In [5]:
# Processing - 
calls = (
    temp_calls.vstack(temp_abbv)
    .with_columns(
        pl.when(pl.col('source').is_in(['FRX','AEM'])).then(pl.lit(5)).otherwise(pl.lit(1)).alias('Salesforce_id')
    )
)

temp1 = (
    calls
    .filter(pl.col('call_week')<=num_weeks_calls)
    .select(['CallID','AttendeeIID','CallDate','CallType','SalesRepIID','Salesforce_id'])
    .join(roster,on='SalesRepIID',how='left')
    .drop('SalesRepIID')
    .with_columns(
        pl.col('CallID').cast(pl.Int64),
        pl.lit('LINZESS').alias('P1'),
        pl.lit('\\N').alias('P2'),
        pl.lit('\\N').alias('P3'),
        pl.col('CallDate').cast(pl.Date)
    )
    .select('CallID','AttendeeIID','Salesforce_id','CallDate','CallType','Rep','P1','P2','P3')
)
temp1.columns = ['CallId','DoctorId','Salesforce_id','Date','CallType','Rep','P1','P2','P3']

In [6]:
#Exporting Feeds-
OUT = 's3://vortex-staging-a65ced90/BIT/output/Doc360/'
temp1.to_pandas().to_csv(f'{OUT}Doc360_CallActivity_Feed.txt', sep='|')
print('Doc360_CallActivity_Feed Exported !')

Doc360_CallActivity_Feed Exported !
